# Big Data for Health (CSE6250) 
Goal: Sepsis prediction using MIMIC III Data

Author: Zhensheng Wang
         
Created: 10/11/2021



In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from tableone import TableOne
import os
os.environ["GOOGLE_APPLICATION_CREDENTIALS"]= os.path.join(os.getcwd(), "bdfh.json")
from google.cloud import bigquery
bqclient = bigquery.Client()

## Query the data

In [ ]:
# Download query results. (change to your own query code)
query_string = """
SELECT *
FROM `cdcproject.BDFH.Sepsis_demographic`
"""

query_string2 = """
SELECT *
FROM `cdcproject.BDFH.Nonsepsis_demographic`
"""

sepsis = bqclient.query(query_string).result().to_dataframe()
nonsepsis = bqclient.query(query_string2).result().to_dataframe()


# Copy the dataframe

In [ ]:
print(f"Number of sepsis patients: {len(sepsis)}")
print(f"Number of control patients: {len(nonsepsis)}")
# sepsis.head(5)

df_sepsis = sepsis.copy()
df_nonsepsis = nonsepsis.copy()

## Clean the data

In [ ]:
def data_clean(df):

    # race recode
    cond_white = df['ETHNICITY'].str.contains('WHITE')
    cond_black = df['ETHNICITY'].str.contains('BLACK')
    cond_asian = df['ETHNICITY'].str.contains('ASIAN')
    cond_hispa = df['ETHNICITY'].str.contains('HISPANIC')

    df.loc[cond_white, 'ETHNICITY'] = 'WHITE'
    df.loc[cond_black, 'ETHNICITY'] = 'BLACK'
    df.loc[cond_asian, 'ETHNICITY'] = 'ASIAN'
    df.loc[cond_hispa, 'ETHNICITY'] = 'HISPANIC'
    df.loc[~(cond_white | cond_black | cond_asian | cond_hispa), 'ETHNICITY'] = 'OTHER'

    df['ETHNICITY'] = df['ETHNICITY'].apply(lambda x: x[0] + x[1:].lower())

    # marital status recode
    cond_other_marital = df['MARITAL_STATUS'].str.contains('SEPARATED|LIFE PARTNER', na = False)
    cond_unknown_marital = df['MARITAL_STATUS'].str.contains('UNKNOWN', na = False) | df['MARITAL_STATUS'].isna()

    df.loc[cond_other_marital, 'MARITAL_STATUS'] = 'OTHER'
    df.loc[cond_unknown_marital, 'MARITAL_STATUS'] = 'UNKNOWN'
    
    df['MARITAL_STATUS'] = df['MARITAL_STATUS'].apply(lambda x: x[0] + x[1:].lower())
    df['gender'] = df['gender'].apply(lambda x: 'Female' if x == 'F' else 'Male')
   
    return df

df_sepsis = data_clean(df_sepsis)
df_nonsepsis = data_clean(df_nonsepsis)

df_sepsis['Sepsis'] = 1
df_nonsepsis['Sepsis'] = 0
df_table1 = pd.concat((df_sepsis, df_nonsepsis), 0).reset_index(drop=True)

## Table 1 Descriptive statistics

In [73]:
columns = ['ETHNICITY', 'gender', 'INSURANCE', 'MARITAL_STATUS', 'los']
categorical = ['ETHNICITY', 'gender', 'INSURANCE', 'MARITAL_STATUS']
order = {
    'ETHNICITY': ['White', 'Black', 'Hispanic', 'Asian', 'Other'],
    'MARITAL_STATUS': ['Single', 'Married', 'Divorced', 'Widowed', 'Other', 'Unknown']
    }
label = {
    'los': 'Length of stay (days)',
    'ETHNICITY': 'Race/Ethnicity',
    'MARITAL_STATUS': 'Marital status',
    'gender': 'Gender',
    'INSURANCE': 'Insurance'
}

t1 = TableOne(
    df_table1, 
    columns = columns, 
    categorical = categorical,
    nonnormal = ['los'],  
    groupby = 'Sepsis', 
    limit = 6, 
    order = order,
    pval = False,
    isnull = False,
    rename = label
)

t1
# print(t1.tabulate(tablefmt="latex"))


Grouped by Sepsis                                 
                                                           Overall               0                1
n                                                            47871           42969             4902
Race/Ethnicity, n (%)                 White           33193 (69.3)    29624 (68.9)      3569 (72.8)
                                      Black             4066 (8.5)      3563 (8.3)       503 (10.3)
                                      Hispanic          1727 (3.6)      1567 (3.6)        160 (3.3)
                                      Asian             1733 (3.6)      1570 (3.7)        163 (3.3)
                                      Other            7152 (14.9)     6645 (15.5)       507 (10.3)
Gender, n (%)                         Female          20988 (43.8)    18809 (43.8)      2179 (44.5)
                                      Male            26883 (56.2)    24160 (56.2)      2723 (55.5)
Insurance, n (%)                      Government        1620 (3.4)      1505 (3.5)        115 (2.3)
                                      Medicaid          4624 (9.7)      4163 (9.7)        461 (9.4)
                                      Medicare        21270 (44.4)    18253 (42.5)      3017 (61.5)
                                      Private         19756 (41.3)    18479 (43.0)      1277 (26.1)
                                      Self Pay           601 (1.3)       569 (1.3)         32 (0.7)
Marital status, n (%)                 Single          10175 (21.3)     8876 (20.7)      1299 (26.5)
                                      Married         19100 (39.9)    16920 (39.4)      2180 (44.5)
                                      Divorced          2454 (5.1)      2121 (4.9)        333 (6.8)
                                      Widowed          5587 (11.7)     4803 (11.2)       784 (16.0)
                                      Other              431 (0.9)       367 (0.9)         64 (1.3)
                                      Unknown         10124 (21.1)     9882 (23.0)        242 (4.9)
Length of stay (days), median [Q1,Q3]               7.0 [4.0,15.0]  7.0 [4.0,14.0]  17.0 [8.0,35.0]